## Difficult estimation

In [1]:
import numpy as np
import random
from pysat.solvers import Solver
from collections import deque
from scipy.ndimage import label
from scipy.spatial import distance
import matplotlib.pyplot as plt
import itertools 
import z3
import json

In [7]:
dataset = np.load("star_battle_dataset.npz")
puzzles = dataset["puzzles"]
solutions = dataset["solutions"]

In [21]:
# Show 5 puzzles
for i in range(len(puzzles)):
    print(f"Showing puzzle {i+1}/{len(puzzles)}")
    print(puzzles[i])
    print(solutions[i])

Showing puzzle 1/5
[[0 0 0 0 9 9 9 1 1 1]
 [0 0 0 0 9 9 9 1 1 1]
 [0 2 2 0 9 1 1 1 1 1]
 [2 2 2 2 1 1 1 1 1 1]
 [5 2 5 8 7 7 7 1 1 4]
 [5 5 5 8 7 7 7 7 4 4]
 [5 5 5 8 7 7 7 6 4 4]
 [5 5 5 8 8 8 6 6 6 4]
 [5 5 5 5 6 6 6 6 6 3]
 [5 5 5 6 6 6 6 3 3 3]]
[[0 0 1 0 0 0 1 0 0 0]
 [1 0 0 0 1 0 0 0 0 0]
 [0 0 1 0 0 0 1 0 0 0]
 [1 0 0 0 0 0 0 0 1 0]
 [0 0 0 1 0 1 0 0 0 0]
 [0 1 0 0 0 0 0 0 1 0]
 [0 0 0 1 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 1]
 [0 1 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 1]]
Showing puzzle 2/5
[[2 2 2 1 1 1 0 0 0 0]
 [2 2 2 1 1 1 0 0 0 0]
 [2 2 2 1 1 1 0 0 0 5]
 [7 1 1 1 1 1 0 0 5 5]
 [7 4 1 1 1 9 9 5 5 5]
 [7 4 8 8 8 3 9 9 5 5]
 [7 4 3 3 3 3 9 9 9 9]
 [7 4 4 3 3 3 9 9 9 9]
 [4 4 4 4 3 6 6 9 6 6]
 [4 4 4 4 4 6 6 6 6 6]]
[[0 1 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 1 0 0]
 [0 0 1 0 0 0 0 0 0 1]
 [1 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 1 0]
 [0 0 1 0 1 0 0 0 0 0]
 [1 0 0 0 0 0 0 1 0 0]
 [0 0 0 1 0 1 0 0 0 0]
 [0 1 0 0 0 0 0 0 1 0]
 [0 0 0 1 0 0 1 0 0 0]]
Showing puzzle 3/5
[[1 0 0 0 0 4 4 4 4

In [ ]:
def classify_difficulty(puzzle, solution):
    #Classifies a Star Battle puzzle into Normal, Hard, or Extreme
    placed_stars = np.zeros_like(puzzle)  # Track placed stars
    definite_zeros = np.zeros_like(puzzle)  # Track definite non-star cells
    num_regions = 10


    if normal_solver(puzzle, solution, placed_stars, definite_zeros):
        return "Normal"
    elif hard_solver(puzzle, solution, placed_stars, definite_zeros):
        return "Hard"
    else:
        return "Extreme"

def normal_solver(puzzle, solution, placed_stars, definite_zeros):
    #attempts to solve using normal heuristics
    for _ in range(20):  # Limit iterations to simulate human solving
        made_progress = False

        made_progress |= apply_three_cell_straight_lines(puzzle, solution, placed_stars, definite_zeros)
        made_progress |= apply_straight_lines(puzzle, solution, placed_stars, definite_zeros)
        made_progress |= apply_t_shapes(puzzle, solution, placed_stars, definite_zeros)
        made_progress |= apply_l_shapes(puzzle, solution, placed_stars, definite_zeros)
        made_progress |= apply_pbdq_shapes(puzzle, solution, placed_stars, definite_zeros)
        made_progress |= apply_offset_stack(puzzle, solution, placed_stars, definite_zeros)        
        made_progress |= apply_simple_center_eliminations(puzzle, solution, placed_stars, definite_zeros)

        if np.array_equal(placed_stars, solution):
            return True

        if not made_progress:
            break  # No more easy deductions, try hard

    return np.array_equal(placed_stars, solution)  # Check if puzzle is fully solved

def hard_solver(puzzle, solution, placed_stars, definite_zeros):
    #attempts solve using normal and hard heuristics
    for _ in range(10):  # limited iterations
        made_progress = False

        made_progress |= apply_rule_of_crowding(puzzle, solution, placed_stars, definite_zeros)
        made_progress |= apply_rule_of_four_squares(puzzle, solution, placed_stars, definite_zeros)
        made_progress |= apply_rule_of_clumps(puzzle, solution, placed_stars, definite_zeros)
        made_progress |= normal_solver(puzzle,solution, placed_stars, definite_zeros)

        if not made_progress:
            break  

    return np.array_equal(placed_stars, solution)  # Check if puzzle is fully solved

# Placeholder functions for specific heuristics (implement detection logic)
def apply_three_cell_straight_lines(puzzle, solution, placed_stars, definite_zeros):
    # Identify 3-cell straight line patterns with stars on both ends
    return False

def apply_straight_lines(puzzle, solution, placed_stars, definite_zeros):
    # definite zeros the whole line
    return False  

def apply_t_shapes(puzzle, solution, placed_stars, definite_zeros):
    # Identify and apply T-shape placement rules
    return False  

def apply_l_shapes(puzzle, solution, placed_stars, definite_zeros):
    # Identify and apply L-shape placement rules
    return False  

def apply_pbdq_shapes(puzzle, solution, placed_star, definite_zeross):
    # Identify and apply P/B/D/Q shape placement rules
    return False  

def apply_offset_stack(puzzle, solution, placed_stars,definite_zeros):
    return False

def apply_simple_center_eliminations(puzzle, solution, placed_stars):
    # Apply eliminations based on 6-cell rectangles and 9-cell squares
    return False  

def apply_rule_of_crowding(puzzle, solution, placed_stars):
    # Implement Rule of Crowding logic
    return False  

def apply_rule_of_four_squares(puzzle, solution, placed_stars):
    # Implement Rule of Four-Squares logic
    return False  

def apply_rule_of_clumps(puzzle, solution, placed_stars):
    # Implement Rule of Clumps logic
    return False  



In [ ]:
# Classify each puzzle
difficulty_labels = [classify_difficulty(puzzles[i], solutions[i]) for i in range(len(puzzles))]

# Save labeled data
np.savez_compressed("star_battle_labeled_dataset.npz", puzzles=puzzles, solutions=solutions, labels=np.array(difficulty_labels))
print("Labeled dataset saved successfully!")

In [ ]:
# dataset = np.load("star_battle_dataset.npz")
# puzzles = dataset["puzzles"]
# solutions = dataset["solutions"]

difficulties = []
for i in range(len(puzzles)):
    difficulty_score = classify_difficulty(puzzles[i], solutions[i])
    
    

# Save difficulty labels
print(difficulties)
# np.savez_compressed("star_battle_dataset_with_difficulty.npz", puzzles=puzzles, solutions=solutions, difficulties=difficulties)
# print("Dataset with difficulty scores saved!")


['Hard', 'Hard', 'Hard', 'Hard', 'Hard']
Dataset with difficulty scores saved!
